In [1]:
# %% [markdown]
# # 测试Control Agent - 使用Mock替换所有子Agent

# %%
import sys
import asyncio
from pathlib import Path
import types

# 添加项目路径
sys.path.append(str(Path.cwd().parent))

# %% [markdown]
# ## 1. 在导入Control Agent之前，先创建Mock模块

# %%
# 创建Mock专家类
class MockExpert:
    """通用的Mock专家类"""
    def __init__(self, name="expert"):
        self.name = name
    
    async def analyze(self, params):
        """模拟分析方法"""
        await asyncio.sleep(0.1)  # 模拟处理时间
        
        intent_type = params.get("intent_type", "report")
        entities = params.get("entities", {})
        
        if intent_type == "report":
            return {
                "summary": f"{self.name}的详细分析报告",
                "key_findings": [
                    f"关于{entities.get('target', '目标')}的发现1",
                    f"关于{entities.get('disease', '疾病')}的发现2"
                ],
                "data": f"来自{self.name}的数据分析"
            }
        else:
            return {
                "answer": f"{self.name}的简短回答",
                "evidence": f"基于{self.name}的数据"
            }

# Mock LLM Client
class MockLLMClient:
    """模拟LLM客户端"""
    async def generate(self, prompt: str) -> str:
        import json
        
        if "分析用户的生物医学研究需求" in prompt:
            if "完整调研报告" in prompt or "做一份" in prompt or "报告" in prompt:
                return json.dumps({
                    "intent_type": "report",
                    "confidence": 0.9,
                    "entities": {
                        "target": "PD-1",
                        "disease": "肺癌",
                        "drug": None,
                        "therapy": "免疫治疗"
                    },
                    "relevant_experts": ["literature_expert", "clinical_expert", "patent_expert"],
                    "reasoning": "用户需要完整报告"
                })
            elif "最新" in prompt or "进展" in prompt:
                return json.dumps({
                    "intent_type": "qa_external",
                    "confidence": 0.85,
                    "entities": {
                        "target": "KRAS G12C",
                        "disease": None,
                        "drug": "KRAS G12C抑制剂",
                        "therapy": None
                    },
                    "relevant_experts": ["clinical_expert", "literature_expert"],
                    "reasoning": "用户询问最新进展"
                })
            elif "刚才" in prompt or "提到" in prompt:
                return json.dumps({
                    "intent_type": "qa_internal",
                    "confidence": 0.95,
                    "entities": {
                        "target": None,
                        "disease": None,
                        "drug": None,
                        "therapy": None
                    },
                    "relevant_experts": [],
                    "reasoning": "询问历史对话"
                })
            elif "对比" in prompt or "比较" in prompt:
                return json.dumps({
                    "intent_type": "target_comparison",
                    "confidence": 0.88,
                    "entities": {
                        "target": "PD-1/PD-L1",
                        "disease": None,
                        "drug": None,
                        "therapy": "免疫检查点抑制剂"
                    },
                    "relevant_experts": ["literature_expert", "clinical_expert"],
                    "reasoning": "用户需要对比分析"
                })
                
        elif "基于以下已有的报告内容" in prompt:
            return "根据之前的讨论，PD-1抗体的主要副作用包括免疫相关不良反应。"
        
        elif "整合以下专家分析结果" in prompt:
            if "qa_external" in prompt:
                return "根据最新数据，KRAS G12C抑制剂显示出良好疗效。"
            elif "target_comparison" in prompt:
                return "PD-1和PD-L1的主要区别在于作用机制和适应症范围。"
        
        return "模拟的LLM响应"

# Mock Editor Agent
class MockEditorAgent:
    """Mock Editor Agent"""
    def generate_report(self, agents_results, gene_target, title):
        html = f"""
        <html>
        <head><title>{title}</title></head>
        <body>
            <h1>{title}</h1>
            <h2>目标: {gene_target}</h2>
            <div>基于{len(agents_results)}个专家的分析...</div>
        </body>
        </html>
        """
        return html

# %% [markdown]
# ## 2. 创建Mock模块并注入到sys.modules

# %%
# 创建Mock模块
mock_llm_module = types.ModuleType('agent_core.clients.llm_client')
mock_llm_module.LLMClient = MockLLMClient
sys.modules['agent_core.clients.llm_client'] = mock_llm_module

# 创建Mock专家模块
mock_lit_module = types.ModuleType('agent_core.agents.specialists.literature_expert')
mock_lit_module.LiteratureExpert = lambda: MockExpert("literature_expert")
sys.modules['agent_core.agents.specialists.literature_expert'] = mock_lit_module

mock_clin_module = types.ModuleType('agent_core.agents.specialists.clinical_expert')
mock_clin_module.ClinicalExpert = lambda: MockExpert("clinical_expert")
sys.modules['agent_core.agents.specialists.clinical_expert'] = mock_clin_module

mock_pat_module = types.ModuleType('agent_core.agents.specialists.patent_expert')
mock_pat_module.PatentExpert = lambda: MockExpert("patent_expert")
sys.modules['agent_core.agents.specialists.patent_expert'] = mock_pat_module

mock_mkt_module = types.ModuleType('agent_core.agents.specialists.market_expert')
mock_mkt_module.MarketExpert = lambda: MockExpert("market_expert")
sys.modules['agent_core.agents.specialists.market_expert'] = mock_mkt_module

mock_editor_module = types.ModuleType('agent_core.agents.specialists.editor_expert')
mock_editor_module.EditorAgent = MockEditorAgent
sys.modules['agent_core.agents.specialists.editor_expert'] = mock_editor_module

# 创建Mock prompts模块
mock_prompts_module = types.ModuleType('agent_core.prompts.control_prompts')

def get_intent_parsing_prompt(user_input: str) -> str:
    return f"分析用户的生物医学研究需求，提取实体。\n用户输入: {user_input}"

def get_memory_search_prompt(query: str, context: str) -> str:
    return f"基于以下已有的报告内容和对话历史，回答用户的问题。\n{context}\n问题: {query}"

def get_expert_aggregation_prompt(results: dict, intent_type: str, query: str) -> str:
    return f"整合以下专家分析结果。意图类型: {intent_type}\n结果: {results}\n问题: {query}"

mock_prompts_module.get_intent_parsing_prompt = get_intent_parsing_prompt
mock_prompts_module.get_memory_search_prompt = get_memory_search_prompt
mock_prompts_module.get_expert_aggregation_prompt = get_expert_aggregation_prompt
sys.modules['agent_core.prompts.control_prompts'] = mock_prompts_module

print("✅ Mock模块已注入")

# %% [markdown]
# ## 3. 现在可以安全导入Control Agent

# %%
# 导入Control Agent和相关类
from agent_core.agents.control_agent import ControlAgent, IntentType
from agent_core.state_machine.graph_definition import create_workflow
from agent_core.state_machine.graph_runner import WorkflowRunner, process_query

print("✅ Control Agent导入成功")

# %% [markdown]
# ## 4. 测试Control Agent核心功能

# %%
async def test_control_agent():
    """测试Control Agent的核心功能"""
    
    control = ControlAgent()
    
    print("="*60)
    print("测试 Control Agent 核心功能")
    print("="*60)
    
    test_cases = [
        ("帮我做一份PD-1靶点的完整调研报告", "report"),
        ("KRAS G12C抑制剂最新的临床试验进展如何？", "qa_external"),
        ("对比一下PD-1和PD-L1抗体的优劣", "target_comparison"),
        ("刚才提到的副作用具体是什么？", "qa_internal")
    ]
    
    for query, expected_type in test_cases:
        print(f"\n{'='*40}")
        print(f"查询: {query}")
        print(f"期望类型: {expected_type}")
        print("-"*40)
        
        try:
            # 1. 测试意图解析
            parsed_intent = await control.parse_intent(query)
            print(f"✓ 意图解析完成")
            print(f"  - 类型: {parsed_intent.intent_type.value}")
            print(f"  - 置信度: {parsed_intent.confidence}")
            print(f"  - 实体: {parsed_intent.entities.to_dict()}")
            print(f"  - 专家: {parsed_intent.relevant_experts}")
            
            # 2. 根据类型测试不同流程
            if parsed_intent.intent_type == IntentType.QA_INTERNAL:
                # 测试内部QA
                response = await control.handle_internal_qa(parsed_intent)
                print(f"✓ 内部QA响应: {response[:100]}...")
                
            else:
                # 测试数据收集
                data_results = await control.collect_data_from_experts(parsed_intent)
                print(f"✓ 数据收集完成: {len(data_results)}个专家")
                
                if parsed_intent.intent_type == IntentType.REPORT:
                    # 测试报告生成
                    report = await control.generate_report(data_results, parsed_intent)
                    print(f"✓ 报告生成完成: {len(report)}字符")
                else:
                    # 测试QA整合
                    answer = await control.aggregate_qa_response(data_results, parsed_intent)
                    print(f"✓ QA整合完成: {answer[:100]}...")
            
            # 3. 保存交互
            control.save_interaction(query, {"test": "response"}, parsed_intent.intent_type.value)
            print(f"✓ 交互已保存")
            
        except Exception as e:
            print(f"❌ 错误: {e}")
            import traceback
            traceback.print_exc()

# 运行测试
await test_control_agent()

# %% [markdown]
# ## 5. 测试State Machine工作流

# %%
async def test_workflow():
    """测试State Machine工作流"""
    
    print("\n" + "="*60)
    print("测试 State Machine 工作流")
    print("="*60)
    
    runner = WorkflowRunner()
    
    test_queries = [
        "帮我做一份PD-1靶点在肺癌中的完整调研报告",
        "KRAS G12C抑制剂的最新进展？",
        "刚才提到的内容能详细说说吗？"
    ]
    
    for query in test_queries:
        print(f"\n查询: {query}")
        print("-"*40)
        
        try:
            result = await runner.run(query)
            
            if result["success"]:
                print(f"✓ 成功")
                print(f"  意图: {result['intent']['type']}")
                print(f"  实体: {result['intent']['entities']}")
                
                response = result['response']
                if response.get('type') == 'report':
                    print(f"  报告长度: {len(response.get('html_content', ''))} 字符")
                else:
                    print(f"  响应类型: {response.get('type')}")
            else:
                print(f"❌ 失败: {result.get('error')}")
                
        except Exception as e:
            print(f"❌ 错误: {e}")
            import traceback
            traceback.print_exc()

# 运行测试
await test_workflow()

# %% [markdown]
# ## 6. 测试便捷API

# %%
async def test_process_query():
    """测试便捷API"""
    
    print("\n" + "="*60)
    print("测试便捷API - process_query")
    print("="*60)
    
    query = "PD-1靶点的研究报告"
    
    try:
        result = await process_query(query)
        print(f"查询: {query}")
        print(f"✓ 成功: {result.get('success')}")
        print(f"  意图: {result.get('intent', {}).get('type')}")
        print(f"  响应类型: {result.get('response', {}).get('type')}")
        
    except Exception as e:
        print(f"❌ 错误: {e}")

# 运行测试
await test_process_query()

# %% [markdown]
# ## 7. 测试缓存机制

# %%
async def test_cache():
    """测试缓存机制"""
    
    control = ControlAgent()
    
    print("\n" + "="*60)
    print("测试缓存机制")
    print("="*60)
    
    query = "PD-1靶点的临床试验进展"
    
    # 第一次查询
    print("\n第一次查询...")
    parsed = await control.parse_intent(query)
    
    import time
    start = time.time()
    results1 = await control.collect_data_from_experts(parsed)
    time1 = time.time() - start
    print(f"耗时: {time1:.2f}秒, 结果数: {len(results1)}")
    
    # 第二次查询（应该使用缓存）
    print("\n第二次查询（相同内容）...")
    start = time.time()
    results2 = await control.collect_data_from_experts(parsed)
    time2 = time.time() - start
    print(f"耗时: {time2:.2f}秒, 结果数: {len(results2)}")
    
    if time2 < time1:
        print(f"✓ 缓存生效，加速 {time1/time2:.1f}倍")
    else:
        print("ℹ️ 缓存可能未生效（Mock的延迟太短）")

# 运行测试
await test_cache()

# %%
print("\n" + "="*60)
print("🎉 所有测试完成！")
print("="*60)

✅ Mock模块已注入
✅ Control Agent导入成功
测试 Control Agent 核心功能

查询: 帮我做一份PD-1靶点的完整调研报告
期望类型: report
----------------------------------------
✓ 意图解析完成
  - 类型: report
  - 置信度: 0.9
  - 实体: {'target': 'PD-1', 'disease': '肺癌', 'therapy': '免疫治疗'}
  - 专家: ['literature_expert', 'clinical_expert', 'patent_expert']
✓ literature_expert 完成数据收集
✓ clinical_expert 完成数据收集
✓ patent_expert 完成数据收集
✓ 数据收集完成: 3个专家
✓ 报告生成完成: 246字符
✓ 交互已保存

查询: KRAS G12C抑制剂最新的临床试验进展如何？
期望类型: qa_external
----------------------------------------
✓ 意图解析完成
  - 类型: qa_external
  - 置信度: 0.85
  - 实体: {'target': 'KRAS G12C', 'drug': 'KRAS G12C抑制剂'}
  - 专家: ['clinical_expert', 'literature_expert']
✓ clinical_expert 完成数据收集
✓ literature_expert 完成数据收集
✓ 数据收集完成: 2个专家
✓ QA整合完成: 根据最新数据，KRAS G12C抑制剂显示出良好疗效。...
✓ 交互已保存

查询: 对比一下PD-1和PD-L1抗体的优劣
期望类型: target_comparison
----------------------------------------
✓ 意图解析完成
  - 类型: target_comparison
  - 置信度: 0.88
  - 实体: {'target': 'PD-1/PD-L1', 'therapy': '免疫检查点抑制剂'}
  - 专家: ['literature_expert', 'clinical_